In [176]:
import bs4
import urllib.parse
import csv
import lxml
import re
import os


In [162]:
from urllib.request import urlopen as ureq
from bs4 import BeautifulSoup as soup
from datetime import datetime
from lxml import html, etree

## Fetching the 10Q link

In [163]:
base_url = 'https://www.sec.gov'
cik = "51143"
acc_num = "000005114313000007"
acc_num_index = acc_num[0:10]+"-"+acc_num[10:12]+"-"+acc_num[12:]+"-index.html"
url_rendered = base_url + "/Archives/edgar/data/" + cik + "/" + acc_num +"/" + acc_num_index
form_url = base_url + "/" + cik + "/" + acc_num
print(url_rendered)

https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/0000051143-13-000007-index.html


In [164]:
uCLient = ureq(url_rendered)
page_html=uCLient.read()

In [165]:
page_soup = soup(page_html, 'html.parser')
divs = page_soup.find('table',summary="Document Format Files")
url2=divs.find_all('tr')[1].find_all('td')[2].find('a')['href']

## Fetching tables

In [166]:
my_url2=urllib.parse.urljoin(base_url, url2)
my_url2

'https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/ibm13q3_10q.htm'

In [167]:
uCLient2 = ureq(my_url2)
page_html2=uCLient2.read()
page_soup2 = soup(page_html2, 'html.parser')

In [168]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [169]:
link_list=[]
link_text=[]

for link in page_soup2.find_all('a'):
    href_link=link.get('href')
    href_text=link.get_text()
    if href_link is not None:
        href_link=str(href_link).strip('#')
        is_Exists = page_soup2.find("a",{'name':href_link})
        if (is_Exists is not None):
            link_list.append(href_link)
            link_text.append(href_text)
print("NEw list",link_list)

NEw list ['Part1FinInfo', 'ConsolidatedFS', 'ConsolEarnings', 'ConsolBS', 'ConsolCF', 'NotesToConsolFS', 'Controls', 'OtherInfo', 'LegalProceedings', 'Unregistered', 'Exhibits']


In [170]:
def checktag(param):
    flag = "false"
    datatabletags = ["background", "bgcolor", "background-color"]
    for x in datatabletags:
        if x in param:
            flag = "true"
    return flag

In [181]:
for item in range(0, (len(link_list)-1)):
    next2 = item + 1
    item1=link_list[item]
    item2=link_list[next2]
    first = '<a name="' + item1 +'">'
    last = '<a name="' + item2 +'">'
    print("elements",first,last)
    new_soup = find_between( page_soup2.prettify(), first, last )
    new_bs = soup(new_soup, 'html.parser')
    tables = new_bs.find_all("table")
    if(len(tables)>0):    
        table_all_rows=[]
        for table in tables:
            table_row = table.select('tr')
            tds = table.select('td')
            flag=0
            for td in tds:
                if checktag(str(td.get('style'))) == "true" or checktag(str(td)) == "true":
                    print("got it")
                    flag=1
                    break
            for tr in table_row:
                if(flag==1):
                    table_column = tr.select('font')
                    row = []
                    for k in table_column:
                        k_text = k.text.replace(u'\xa0',u'')
                        k_text = k_text.replace(u'\n',u'')
                        if(k_text!='\n' ):
                            row.append(k_text)
                    table_all_rows.append(row)
            if(flag==1):
                dirname = './Part1/'+cik+ '/'+acc_num+'/'
                if not os.path.exists(dirname):
                    os.makedirs(dirname)
                try:
                    with open((dirname+item1+'.csv'),"a",newline='') as my_csv:
                        csvWriter = csv.writer(my_csv,delimiter=',')
                        try:
                            csvWriter.writerows(table_all_rows) 
                        except:
                            print("no")
                except:
                    print("file open error")

elements <a name="Part1FinInfo"> <a name="ConsolidatedFS">
elements <a name="ConsolidatedFS"> <a name="ConsolEarnings">
elements <a name="ConsolEarnings"> <a name="ConsolBS">
got it
got it
elements <a name="ConsolBS"> <a name="ConsolCF">
got it
got it
elements <a name="ConsolCF"> <a name="NotesToConsolFS">
elements <a name="NotesToConsolFS"> <a name="Controls">
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it